In [50]:
!pip install pymilvus

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [51]:
from pymilvus import connections, db, Collection , utility

In [52]:
connections.add_connection(
    rag_conn = {
        "host":"localhost",
        "port":19530,
        "username":"",
        "password":""
        }
)

conn_name = 'rag_conn'
db_name = 'rag_db'

connections.connect(conn_name)
connections.list_connections()

current_dbs = db.list_database(using=conn_name)

if db_name not in current_dbs:
    print(f"{db_name} not in current dbs list, creating one!!!")
    resume_db = db.create_database(db_name, using=conn_name)

db.using_database(db_name, using= conn_name)

rag_db not in current dbs list, creating one!!!


In [53]:
from pymilvus import FieldSchema, CollectionSchema, Collection, utility, DataType

chunk_id = FieldSchema(
    name='chunk_id',
    dtype=DataType.INT64,
    is_primary = True,
    max_length = 32
)

rag_text_field = FieldSchema(
    name='rag_text_field',
    dtype=DataType.VARCHAR,
    max_length = 2048
)

rag_embedding_field = FieldSchema(
    name ='rag_embedding_field',
    dtype=DataType.FLOAT_VECTOR,
    dim = 384
)

rag_schema = CollectionSchema(
    fields=[chunk_id, rag_text_field, rag_embedding_field],
    description='rag_schema',
    enable_dynamic_field = True
)

collection_name = 'rag_collection'

rag_collection = Collection(
    name='rag_collection',
    schema=rag_schema,
    using= conn_name,
    shard_nums = 2
)

print("Collections: ", utility.list_collections(using=conn_name))

r_collection = Collection(collection_name, using=conn_name )
print("Schema: ", r_collection.schema)

Collections:  ['rag_collection']
Schema:  {'auto_id': False, 'description': 'rag_schema', 'fields': [{'name': 'chunk_id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'rag_text_field', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 2048}}, {'name': 'rag_embedding_field', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': True}


### Data Prep For Knowledge Base

In [54]:
# !pip install langchain_community

In [55]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("LLM_Demo.pdf")
pdf_docs = loader.load()

In [56]:
print(pdf_docs)

[Document(metadata={'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'LLM_Demo.pdf', 'file_path': 'LLM_Demo.pdf', 'total_pages': 3, 'format': 'PDF 1.4', 'title': 'LLM_Demo', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Large Language Models, commonly known as LLMs, represent one of the most significant \nmilestones in the evolution of artificial intelligence, allowing machines to understand, reason \nwith, and generate human-like language across an astonishing variety of tasks. These models, \nsuch as GPT, Gemini, Claude, and Mistral, share a common foundation built upon the \nTransformer architecture introduced in 2017 in the groundbreaking paper “Attention Is All You \nNeed.” The Transformer replaced recurrent and convolutional architectures with a mechanism \nknown as self-attention, enabling each word or token in a sequence to interact with every 

In [57]:
!pip install langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


#### Cleaning the text

In [58]:
import re
from langchain_core.documents import Document

cleaned_data = []
def clean_pdf_text(pdf_docs):
    for d in pdf_docs:
        text = d.page_content
        text = re.sub(r'\s+', ' ', text)
        text = text.replace(" \n", " ").replace("\n ", " ").replace("\n", " ")
        text = text.strip()
        cleaned_data.append(Document(page_content = text, metadata = d.metadata))
    return cleaned_data

pdf_docs = clean_pdf_text(pdf_docs)

In [59]:
print(pdf_docs)

[Document(metadata={'producer': 'Skia/PDF m144 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'LLM_Demo.pdf', 'file_path': 'LLM_Demo.pdf', 'total_pages': 3, 'format': 'PDF 1.4', 'title': 'LLM_Demo', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Large Language Models, commonly known as LLMs, represent one of the most significant milestones in the evolution of artificial intelligence, allowing machines to understand, reason with, and generate human-like language across an astonishing variety of tasks. These models, such as GPT, Gemini, Claude, and Mistral, share a common foundation built upon the Transformer architecture introduced in 2017 in the groundbreaking paper “Attention Is All You Need.” The Transformer replaced recurrent and convolutional architectures with a mechanism known as self-attention, enabling each word or token in a sequence to interact with every other token 

In [60]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 32,
    length_function = len,
    separators=["\n\n", "\n", ".", "!", "?", ",", " "]
)

pdf_docs = text_splitter.split_documents(pdf_docs)

rag_text = []
for i in pdf_docs:
    rag_text.append(i.page_content)

print("Total Chunks:", len(rag_text))
print("Sample Chunk Text:", rag_text[1])

Total Chunks: 22
Sample Chunk Text: .” The Transformer replaced recurrent and convolutional architectures with a mechanism known as self-attention, enabling each word or token in a sequence to interact with every other token based on learned relevance scores. During training, the model consumes vast amounts of textual data drawn from the internet, books, and other corpora, transforming it into embeddings—dense numerical vectors that encode meaning, syntax, and context


In [61]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

rag_embeddings = [embedding_model.encode(i) for i in rag_text]
record_ids = [i for i in range(len(rag_text))]

/Users/utkarshsingh/LLM_Foundation_RAG/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [62]:
insert_data = [record_ids, rag_text, rag_embeddings]

i_collection = Collection(collection_name, using=conn_name)

mr = i_collection.insert(insert_data)
i_collection.flush()


index_params = {
    'index_type':'IVF_FLAT',
    'metric_type':'L2',
    'params':{"nlist":1024}
}

i_collection.create_index(
    field_name="rag_embedding_field",
    index_params=index_params
)

utility.index_building_progress(collection_name, using=conn_name)

{'total_rows': 22,
 'indexed_rows': 22,
 'pending_index_rows': 0,
 'state': 'Finished'}

### Answering Questions with RAG

In [75]:
search_params = {
    'metric_type':'L2',
    'offset':0,
    'ignore_growing':False,
    'params':{"nprobe":20}
}


query = "what is llm used for?"
search_embedding = embedding_model.encode(query)

q_collection = Collection(collection_name, using=conn_name)
q_collection.load()

results = q_collection.search(
    data = [search_embedding],
    param= search_params,
    anns_field="rag_embedding_field",
    limit=4,
    # offset =0,
    expr= None,
    output_fields=['rag_text_field'],
    consistency_level = 'Strong'
)
# print(results)

print("Top Results:", results[0][0])

Top Results: id: 2, distance: 1.1279608011245728, entity: {'rag_text_field': '. The learning process involves predicting the next token in a sequence given its context, an optimization task solved using gradient descent across billions or even trillions of parameters distributed across massive clusters of GPUs or TPUs. Over time, the LLM internalizes statistical relationships and conceptual structures that allow it to generate coherent, contextually relevant responses, even in situations it has never explicitly seen before. Once trained, these models exhibit remarkable versatility'}


#### Creating Prompt for LLM

In [78]:
context = []

for i in range(len(results[0])):
    context.append(results[0][i].entity.get('rag_text_field'))

prompt = (f"Based on the Query by user here : '{query}' you have to answer the question below with this context: '{context}' ")

print(prompt)

Based on the Query by user here : what is llm used for? you have to answer the question below with this context:['. The learning process involves predicting the next token in a sequence given its context, an optimization task solved using gradient descent across billions or even trillions of parameters distributed across massive clusters of GPUs or TPUs. Over time, the LLM internalizes statistical relationships and conceptual structures that allow it to generate coherent, contextually relevant responses, even in situations it has never explicitly seen before. Once trained, these models exhibit remarkable versatility', 'Large Language Models, commonly known as LLMs, represent one of the most significant milestones in the evolution of artificial intelligence, allowing machines to understand, reason with, and generate human-like language across an astonishing variety of tasks. These models, such as GPT, Gemini, Claude, and Mistral, share a common foundation built upon the Transformer arch

### Prompting the LLM

In [84]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(temperature = 0.0,model='llama3.2')

result_complete = llm.invoke(prompt)

print(result_complete)

content='Based on the provided context, Large Language Models (LLMs) are used for a variety of tasks, including:\n\n1. Conversational AI: Chatbots powered by LLMs like GPT or Claude handle millions of daily queries, demonstrating empathy, reasoning, and contextual awareness that approach human quality.\n2. Content generation: Marketers and journalists rely on LLMs to draft narratives, blogs, and reports at unprecedented scale.\n3. Language understanding and reasoning: LLMs exhibit long-range reasoning without sequential bottlenecks typical of RNNs, allowing them to understand and reason with complex language inputs.\n4. Human-like language generation: LLMs can generate coherent, contextually relevant responses, even in situations they have never explicitly seen before.\n\nOverall, LLMs are versatile models that can be applied to a wide range of tasks, from conversational AI to content generation, and are capable of understanding and generating human-like language.' additional_kwargs={}